In [7]:
#pip install beautifulsoup4
#!pip install lxml
#!pip install html5lib
#!pip install requests

<h2>1) Scraping Wikipedia and Preprocessing Data</h2>

In [32]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content)

table = soup.find('table')
rows = table.find_all('tr')
data_df = []
#df = pd.DataFrame(data, columns = ['Postal code', 'Borough','Neighborhood']) 

for row in rows:
    data = row.find_all("td")
    if len(data) > 0:
        if(data[1].get_text() != 'Not assigned\n'):
            replace_str =  data[2].get_text().strip('\n').replace("/", ",")
            data_df.append([data[0].get_text().strip('\n'),data[1].get_text().strip('\n'),data[2].get_text().strip('\n')])

df = pd.DataFrame(data_df, columns = ['Postal Code', 'Borough','Neighborhood']) 

In [33]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [34]:
df.shape

(103, 3)

<h2>2) Merge GeoSpatial data with Postal Code</h2>

In [35]:
geo_data = pd.read_csv("http://cocl.us/Geospatial_data") 
geo_data

geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
df_merge = pd.merge(df, geo_data, on='Postal Code', how='inner')
df_merge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
